In [1]:
import numpy as np
import pandas as pd
import wave
from scipy.io import wavfile
import os
import librosa
from librosa.feature import melspectrogram
import warnings
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from PIL import Image
from uuid import uuid4
import sklearn
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, LSTM, SimpleRNN, Conv1D, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-9gvgieod because the default path (/home/dvm/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
#birds_to_recognise = sorted(shuffle(most_represented_birds)[:20])
birds_to_recognise = ['House Sparrow', 'Common Blackbird', 'Great Tit', 'Common Starling', 'Eurasian Blue Tit', 'Eurasian Tree Sparrow', 'Eurasian Magpie', 'Common Wood Pigeon', 'European Robin', 'Common House Martin', 'Common Swift', 'Carrion Crow', 'Common Chaffinch', 'Eurasian Collared Dove', 'European Goldfinch', 'Great Spotted Woodpecker', 'Barn Swallow', 'Eurasian Jay', 'Rock Dove', 'Eurasian Bullfinch']
print(birds_to_recognise)

['House Sparrow', 'Common Blackbird', 'Great Tit', 'Common Starling', 'Eurasian Blue Tit', 'Eurasian Tree Sparrow', 'Eurasian Magpie', 'Common Wood Pigeon', 'European Robin', 'Common House Martin', 'Common Swift', 'Carrion Crow', 'Common Chaffinch', 'Eurasian Collared Dove', 'European Goldfinch', 'Great Spotted Woodpecker', 'Barn Swallow', 'Eurasian Jay', 'Rock Dove', 'Eurasian Bullfinch']


In [3]:
classes_to_predict = birds_to_recognise

In [4]:
model = keras.models.load_model("/home/dvm/Desktop/nachtall_final.h5")


In [5]:
def predict_on_melspectrogram(song_sample, sample_length):
    N_mels=216

    if len(song_sample)>=sample_length:
        mel = melspectrogram(song_sample, n_mels=N_mels, sr=22050)
        db = librosa.power_to_db(mel)
        normalised_db = sklearn.preprocessing.minmax_scale(db)
        db_array = (np.asarray(normalised_db)*255).astype(np.uint8)
        prediction = model.predict(np.array([np.array([db_array, db_array, db_array]).T]))
        predicted_bird = classes_to_predict[np.argmax(prediction)]
        if predicted_bird=="Barn Swallow":
            predicted_bird="nocall"
   #     db_image =  Image.fromarray(np.array([db_array, db_array, db_array]).T)
   #     demo_img = db_image
   #     plt.imshow(demo_img)
   #     plt.show()
        return predicted_bird
    else:
        return "nocall"
    


In [7]:
wav_df = pd.read_csv('/home/dvm/Desktop/dataframes/wav_dataframe.csv')

classification = []

for index, row in wav_df.iterrows():
    print(row[0])
    chunk_filename = row[0]
    wave_data, wave_rate = librosa.load(chunk_filename, sr=22050)
    sample_length = 5*wave_rate
    song_sample = np.array(wave_data[0:sample_length])
    predicted_bird = predict_on_melspectrogram(song_sample, sample_length)
    classification.append(predicted_bird)
    print(predicted_bird)


classification_df = pd.DataFrame(classification, columns=['Classification'])
class_new = pd.concat([wav_df, classification_df], axis=1, ignore_index=True )
class_new.to_csv('/home/dvm/Desktop/dataframes/class_df.csv', index=False)







/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N0.wav
Common Swift
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N1.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N2.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N3.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N4.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N5.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N6.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N7.wav
Common Wood Pigeon
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N8.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N9.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N10.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N11.wav
Common Wood Pigeon
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_AM_N12.wav
Barn Swallow
/home/dvm/Desktop/recordings/2021_06_24-04_00_01_

In [ ]:
def predict_submission(df, audio_file_path):
        
    loaded_audio_sample = []
    previous_filename = ""
    wave_data = []
    wave_rate = None
    sample_length = None
    
    for idx,row in df.iterrows():
        #I added this exception as I've heard that some files may be corrupted.
        try:
            if previous_filename == "" or previous_filename!=row.audio_id:
                filename = '{}/{}.mp3'.format(audio_file_path, row.audio_id)
                wave_data, wave_rate = librosa.load(filename, sr=22050)
                sample_length = 5*wave_rate
            previous_filename = row.audio_id

            #basically allows to check if we are running the examples or the test set.
            if "site" in df.columns:
                if row.site=="site_1" or row.site=="site_2":
                    song_sample = np.array(wave_data[int(row.seconds-5)*wave_rate:int(row.seconds)*wave_rate])
                elif row.site=="site_3":
                    #for now, I only take the first 5s of the samples from site_3 as they are groundtruthed at file level
                    song_sample = np.array(wave_data[0:sample_length])
            else:
                #same as the first condition but I isolated it for later and it is for the example file
                song_sample = np.array(wave_data[int(row.seconds-5)*wave_rate:int(row.seconds)*wave_rate])
            
            predicted_bird = predict_on_melspectrogram(song_sample, sample_length)
            df.at[idx,"birds"] = predicted_bird
        except:
            df.at[idx,"birds"] = "nocall"
    return df

Below, We can test our prediction function using the examples provided.

In [ ]:
audio_file_path = "/home/dvm/Schreibtisch/nachtall_Code/Data_training/new_songs"
example_df = pd.read_csv("/home/dvm/Schreibtisch/nachtall_Code/Data_training/nachtall_train_val.csv")
#Ajusting the example filenames and creating the audio_id column to match with the test file.

if os.path.exists(audio_file_path):
    example_df = predict_submission(example_df, audio_file_path)
example_df

In [ ]:
example_df.to_csv("class.csv")

In [ ]:
test_file_path = "/test_audio"
test_df = pd.read_csv("test.csv")
submission_df = pd.read_csv("sample_submission.csv")

if os.path.exists(test_file_path):
    submission_df = predict_submission(test_df, test_file_path)

submission_df[["row_id","birds"]].to_csv('submission.csv', index=False)
submission_df.head()